# Fetch List of Cases

In [71]:
import requests
import pandas as pd
import numpy as np
import re
import pickle

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm

In [ ]:
data = pd.DataFrame()

In [ ]:
for i in tqdm(range(49,0,-1)):
    data_page = []
    # set the URL
    url = 'http://sipp.pn-jakartapusat.go.id/http://sipp.pn-jakartapusat.go.id/list_perkara/page/'+str(i)+'/OXB2Nkc3U0lzUVoza1pGcjMrNUx6QkZyRU5ZU1cxekczRnYrcGY0ZDN3elVpKzFjeDJjWXAzOWRTOU9ENy81ejFFUlFSRzkxWURhaWVzNWVKYUdYTGc9PQ==/key/col/2:147'
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(20)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    # parsing data row by row
    table_rows = soup.find_all('tr')
    for row in table_rows:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') if ele.get_text() != '[detil]' else ele.find('a')['href']  for ele in cols]
        data_page.append(cols)
    
    page_table = pd.DataFrame(data_page[1:-1],columns=data_page[0]).set_index('Nomor Perkara').drop('No',axis=1)
    data = data.append(page_table)

In [ ]:
# Remove duplicates by case ID
data = data[~data.index.duplicated(keep='first')]

In [ ]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'wb') as f:
    pickle.dump(data, f)

# Crawled data to CSV
data.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.csv')

# Fetch Case Details

## Fetch Judge and Clerk Name

In [ ]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [ ]:
case_list = list(datax.index.unique())
len(case_list)

In [ ]:
data_hakim = pd.DataFrame()
data_panitera = pd.DataFrame()
data_sidang_pertama = pd.DataFrame()

In [ ]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
    url_hakim = url.replace("show_detil","show_penetapan")
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url_hakim)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()
    
    # Tabel Hakim
    kolom_tabel_hakim = ['Nomor Perkara','Tanggal Penetapan', 'Nama Hakim/Majelis Hakim', 'Posisi', 'Aktif']
    hakim_kasus = []
    table_hakim = soup.find_all('div',class_='cssTable')[0].find_all('tr')
    for row in table_hakim:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        hakim_kasus.append(cols)
    
    page_table = pd.DataFrame(hakim_kasus[1:],columns=kolom_tabel_hakim).set_index('Nomor Perkara')
    data_hakim = data_hakim.append(page_table)
    
    # Tabel Panitera
    kolom_tabel_panitera = ['Nomor Perkara','Tanggal Penetapan', 'Nama Panitera Pengganti', 'Aktif']
    panitera_kasus = []
    table_panitera = soup.find_all('div',class_='cssTable')[1].find_all('tr')
    for row in table_panitera:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        panitera_kasus.append(cols)
    
    table_panitera = pd.DataFrame(panitera_kasus[1:],columns=kolom_tabel_panitera).set_index('Nomor Perkara')
    data_panitera = data_panitera.append(table_panitera)
    
    # Tabel Hari Sidang Pertama
    kolom_tabel_pertama = ['Nomor Perkara','Tanggal Penetapan', 'Tanggal Sidang Pertama']
    sidang_pertama_kasus = []
    table_sidang_pertama = soup.find_all('div',class_='cssTable')[2].find_all('tr')
    for row in table_sidang_pertama:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        sidang_pertama_kasus.append(cols)
    
    table_sidang_pertama = pd.DataFrame(sidang_pertama_kasus[1:],columns=kolom_tabel_pertama).set_index('Nomor Perkara')
    data_sidang_pertama = data_sidang_pertama.append(table_sidang_pertama)    

In [ ]:
data_hakim.head()

In [ ]:
data_panitera.tail()

In [ ]:
data_sidang_pertama.tail()

In [ ]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.pickle', 'wb') as f:
    pickle.dump(data_hakim, f)
    
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_clerk_list.pickle', 'wb') as f:
    pickle.dump(data_panitera, f)
    
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_first_court_day.pickle', 'wb') as f:
    pickle.dump(data_sidang_pertama, f)

# Crawled data to CSV
data_hakim.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.csv')
data_panitera.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_clerk_list.csv')
data_sidang_pertama.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_first_court_day_list.csv')

## Fetch General Info of Case

In [ ]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [ ]:
case_list = list(datax.index.unique())
len(case_list)

In [ ]:
general_info_table = pd.DataFrame()

In [ ]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
 
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()    
      
    # Info Umum
    general_info = soup.find_all('table', id='infoPerkara')[0].find_all('tr')
    
    # Skip the table-inside-table situation
    test = []
    for i in general_info:
        if len(i.find_parents("tr")) == 0:
            if i.div is None:
                test.append(i)
    kolom_general_info = ['Nomor Perkara','Atribut','Nilai']
    
    info = []
    for row in test:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        info.append(cols)

    page_info = pd.DataFrame(info,columns=kolom_general_info).set_index('Nomor Perkara')
    general_info_table = general_info_table.append(page_info)

In [ ]:
len(list(general_info_table.index.unique()))

In [ ]:
# Transpose it!
df = general_info_table
df.reset_index(inplace=False)
transposed_df = df.pivot(columns='Atribut',values='Nilai')
transposed_df

In [ ]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/general_info_table.pickle', 'wb') as f:
    pickle.dump(transposed_df, f)

transposed_df.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/general_info_table.csv')

# Fetch Sentence

In [ ]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [ ]:
case_list = list(datax.index.unique())
len(case_list)

In [ ]:
sentence_table = pd.DataFrame()

In [ ]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
    url = url.replace("show_detil","show_putusan")
 
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()    
      
    # Info Umum
    general_info = soup.find_all('table', id='infoPerkara')[0].find_all('tr')
    
    # Skip the table-inside-table situation
    test = []
    for i in general_info:
        if len(i.find_parents("tr")) == 0:
            if i.div is None:
                test.append(i)
                
    kolom_general_info = ['Nomor Perkara','Atribut','Nilai']
    
    info = []
    for row in test:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        info.append(cols)
    
    info = pd.DataFrame(info)
    info = info.iloc[:,0:3]
    info.columns = kolom_general_info
    page_info = info.set_index('Nomor Perkara')

    sentence_table = sentence_table.append(page_info)

In [ ]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/sentence_table.pickle', 'wb') as f:
    pickle.dump(sentence_table, f)

sentence_table.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/sentence_table.csv')

In [2]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/sentence_table.pickle', 'rb') as f:
    datax = pickle.load(f)

In [8]:
datax.tail(15)

,Atribut,Nilai
Nomor Perkara,,
00013/PID.B/TPKOR/2011/PN.JKT.PST,Tanggal Putusan,"Kamis, 14 Jul. 2011"
00013/PID.B/TPKOR/2011/PN.JKT.PST,Amar Putusan,MENGADILI : \n \n 1. Menyatakan Terdakwa H. ...
00013/PID.B/TPKOR/2011/PN.JKT.PST,Kirim Salinan Putusan Kepada Penyidik,
00013/PID.B/TPKOR/2011/PN.JKT.PST,Tanggal Minutasi,"Kamis, 22 Sep. 2011"
00013/PID.B/TPKOR/2011/PN.JKT.PST,Keterangan,
00014/PID.B/TPKOR/2011/PN.JKT.PST,Tanggal Putusan,"Kamis, 16 Jun. 2011"
00014/PID.B/TPKOR/2011/PN.JKT.PST,Amar Putusan,MENGADILI : \n 1.Menyatakan Terdakwa I Agus Co...
00014/PID.B/TPKOR/2011/PN.JKT.PST,Kirim Salinan Putusan Kepada Penyidik,
00014/PID.B/TPKOR/2011/PN.JKT.PST,Tanggal Minutasi,"Kamis, 11 Agu. 2011"


In [45]:
datax['Atribut'].unique()

array(['Tanggal Putusan', 'Status Putusan', 'Amar Putusan',
       'Pemberitahuan Putusan', 'Menerima Putusan',
       'Kirim Salinan Putusan', 'Kirim Salinan Putusan Kepada Penyidik',
       'Tanggal Minutasi', 'Keterangan', None, '\n NO. \n', '\n \xa0 \n'], dtype=object)

In [56]:
a = [2,3,4,5]
3 in a

True

In [60]:
datax = datax[datax['Atribut'].isin(['Tanggal Putusan', 'Status Putusan', 'Amar Putusan','Pemberitahuan Putusan', 'Menerima Putusan','Kirim Salinan Putusan', 'Kirim Salinan Putusan Kepada Penyidik', 'Tanggal Minutasi', 'Keterangan'])]
datax = datax.reset_index().drop_duplicates(['Nomor Perkara','Atribut']).pivot(index='Nomor Perkara', columns='Atribut', values='Nilai')

In [62]:
datax.head()

Atribut,Amar Putusan,Keterangan,Kirim Salinan Putusan,Kirim Salinan Putusan Kepada Penyidik,Menerima Putusan,Pemberitahuan Putusan,Status Putusan,Tanggal Minutasi,Tanggal Putusan
Nomor Perkara,,,,,,,,,
00001/PID.B/TPKOR/2011/PN.JKT.PST,MENGADILI : \n 1. Menyatakan keberatan Tim Pen...,bpp,None,,None,None,\n,"Selasa, 19 Apr. 2011","Senin, 21 Feb. 2011"
00001/PID.B/TPKOR/2012/PN.JKT.PST,MENGADILI : \n \n 1. Menyatakan terdakwa Ir....,bpp,None,,None,None,\n,"Selasa, 19 Jun. 2012","Rabu, 02 Mei 2012"
00002/PID.B/TPKOR/2011/PN.JKT.PST,MENGADILI : \n 1. Menyatakan Terdakwa Drs.Enda...,,None,"Rabu, 20 Apr. 2011",None,None,None,"Senin, 13 Jun. 2011","Senin, 18 Apr. 2011"
00002/PID.B/TPKOR/2012/PN.JKT.PST,MENGADILI: \n 1. Menyatakan terdakwa Bambang T...,bpp,None,,None,None,\n,"Selasa, 12 Jun. 2012","Senin, 14 Mei 2012"
00003/PID.B/TPKOR/2011/PN.JKT.PST,MENGADILI : \n \n 1. Menyatakan Terdakwa Cep...,bpp,None,,None,None,\n,"Kamis, 25 Agu. 2011","Selasa, 28 Jun. 2011"


In [63]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/sentence_table.pickle', 'wb') as f:
    pickle.dump(datax, f)

datax.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/sentence_table.csv')

In [70]:
datax.loc['00031/PID.B/TPKOR/2012/PN.JKT.PST']['Amar Putusan']

'MENGADILI : \n 1. Menyatakan terdakwa T.Helmy Azwari terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana korupsi secara bersama-sama sebagai perbuatan berlanjut sebagaimana diatur dalam Pasal 3 UU No.15/2002. \n 2. Menjatuhkan pidana terhadap terdakwa selama 4 tahun 6 bulan dan denda Rp.350.000.000,- Apabila denda tidak dibayar diganti pidana kurungan 3 bulan. \n 3. Menghukum terdakwa membayar uang pengganti Rp.235.623.287,- Jika terpidana tidak membayar dalam waktu 1 bulan setelah putusan ini berkekuatan hukum tetap, maka harta bendanya disita dan dilelang oleh Jaksa. Apabila harta bendanya tidak mencukupi, maka dipidana penjara selama 3 bulan. \n 4. Menetapkan masa penahanan yang dijalankan terdakwa dikurangkan seluruhnya dari jumlah pidana yang dijatuhkan. \n 5. Memerintahkan agar terdakwa tetap berada dalam tahanan. \n 6. Memerintahkan barang bukti digunakan dalam perkara lain. \n 7. Menetapkan biaya perkara Rp.10.000,- dibebankan kepada terdakwa.'

# Fetch Data Terdakwa

In [106]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [107]:
case_list = list(datax.index.unique())
len(case_list)

964

In [108]:
terdakwa_table = pd.DataFrame()
jaksa_table = pd.DataFrame()

In [109]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
 
    driver = webdriver.Chrome()
    driver.implicitly_wait(15)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()
    
    jaksa_info = soup.find_all('table', id='tableinfo')[1].find_all('tr')
#     print(jaksa_info)
    
    kolom_jaksa_info = ['Nomor Perkara','No','Nama']
    
    info = []
    for row in jaksa_info:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        info.append(cols)
    
    info = pd.DataFrame(info)
    info = info.iloc[1:,:]
    info.columns = kolom_jaksa_info
    page_info = info.set_index('Nomor Perkara')
    
    jaksa_table = jaksa_table.append(page_info)
    
    
    # Terdakwa
    terdakwa_info = soup.find_all('table', id='tableinfo')[2].find_all('tr')
#     print(terdakwa_info)    
    
    kolom_terdakwa_info = ['Nomor Perkara','No','Nama']
    
    info_t = []
    for row in terdakwa_info:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') for ele in cols]
        cols.insert(0,case)
        info_t.append(cols)
    
    info_t = pd.DataFrame(info_t)
    info_t = info_t.iloc[1:,0:3]
    info_t.columns = kolom_terdakwa_info
    page_info = info_t.set_index('Nomor Perkara')
    
    terdakwa_table = terdakwa_table.append(page_info)   

A Jupyter Widget

In [110]:
jaksa_table

,No,Nama
Nomor Perkara,,
58/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"RONALD F.W., SH."
57/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"RONALD F.W., SH."
61/Pid.Sus-TPK/2019/PN Jkt.Pst,1,FATONI HATAM
60/Pid.Sus-TPK/2019/PN Jkt.Pst,1,FATONI HATAM
38/Pid.Sus-TPK/2019/PN Jkt.Pst,1,Putra Iskandar
42/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"TIMBUL M., SH."
41/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"TIMBUL M., SH."
40/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"TIMBUL M., SH."
39/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"TIMBUL M., SH."


In [111]:
terdakwa_table

,No,Nama
Nomor Perkara,,
58/Pid.Sus-TPK/2019/PN Jkt.Pst,1,ADHI PURNOMO
58/Pid.Sus-TPK/2019/PN Jkt.Pst,2,EKO TRIYANTA
57/Pid.Sus-TPK/2019/PN Jkt.Pst,1,MULYANA
61/Pid.Sus-TPK/2019/PN Jkt.Pst,1,Ezrinal Azis Pgl. AL alias AZIS alias EZ
60/Pid.Sus-TPK/2019/PN Jkt.Pst,1,ZUBAEDI
38/Pid.Sus-TPK/2019/PN Jkt.Pst,1,Bambang Mustaqim
42/Pid.Sus-TPK/2019/PN Jkt.Pst,1,"Dr. dr. Eka Wahyu Kasih, S.Pd., SH., MH., MM"
41/Pid.Sus-TPK/2019/PN Jkt.Pst,1,Bimo Wicaksono
40/Pid.Sus-TPK/2019/PN Jkt.Pst,1,Gompis Lumban Tobing


In [112]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/jaksa_table.pickle', 'wb') as f:
    pickle.dump(jaksa_table, f)

jaksa_table.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/jaksa_table.csv')

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/terdakwa_table.pickle', 'wb') as f:
    pickle.dump(terdakwa_table, f)

terdakwa_table.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/terdakwa_table.csv')